# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [154]:
import pandas as pd
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
insurance_df = pd.read_csv(url)

In [155]:
insurance_df.drop(columns = ['Unnamed: 0'],inplace=True)
insurance_df.rename(columns={col:col.strip().lower().replace(" ","_") for col in insurance_df.columns},inplace=True)
insurance_df['effective_to_date'] = pd.to_datetime(insurance_df['effective_to_date'], format="%m/%d/%y", errors='coerce')
insurance_df['effective_to_date'] = insurance_df['effective_to_date'].dt.strftime("%d/%m/%y")
insurance_df['number_of_open_complaints'].fillna(0,inplace=True)
insurance_df['number_of_open_complaints']=insurance_df['number_of_open_complaints'].astype(int)
insurance_df.drop_duplicates(inplace=True)
insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10873 entries, 0 to 10909
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer                       10873 non-null  object 
 1   state                          10259 non-null  object 
 2   customer_lifetime_value        10873 non-null  float64
 3   response                       10259 non-null  object 
 4   coverage                       10873 non-null  object 
 5   education                      10873 non-null  object 
 6   effective_to_date              10873 non-null  object 
 7   employmentstatus               10873 non-null  object 
 8   gender                         10873 non-null  object 
 9   income                         10873 non-null  int64  
 10  location_code                  10873 non-null  object 
 11  marital_status                 10873 non-null  object 
 12  monthly_premium_auto           10873 non-null  int6

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [156]:
# total_claim_amount > 1000
# response = Yes
task1_df = insurance_df[(insurance_df['total_claim_amount']>1000) & (insurance_df['response'] == 'Yes') ]
task1_df

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,OK31456,California,11009.130490,Yes,Premium,Bachelor,24/01/11,Employed,F,51643,...,0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,24/01/11,Employed,F,51643,...,0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,GW43195,Oregon,25807.063000,Yes,Extended,College,13/02/11,Employed,F,71210,...,1,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,IP94270,Arizona,13736.132500,Yes,Premium,Master,13/02/11,Disabled,F,16181,...,0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,26/01/11,Unemployed,M,0,...,0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,FN44127,Oregon,3508.569533,Yes,Extended,College,05/01/11,Medical Leave,M,20978,...,1,1,Personal Auto,Personal L2,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10373,XZ64172,Oregon,10963.957230,Yes,Premium,High School or Below,08/02/11,Employed,M,55687,...,0,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.800000,Luxury SUV,Medsize,NaN
10487,IX60941,Oregon,3508.569533,Yes,Extended,College,05/01/11,Medical Leave,M,20978,...,1,1,Personal Auto,Personal L3,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10565,QO62792,Oregon,7840.165778,Yes,Extended,College,14/01/11,Employed,M,58414,...,2,1,Personal Auto,Personal L3,Offer2,Agent,1008.000000,NaN,NaN,NaN


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [157]:
task2_df = insurance_df[(insurance_df['response'] == 'Yes')]
display(task2_df.groupby(['gender','policy_type'])['total_claim_amount'].mean())
print('Females are paying more for corporate auto and special auto policy compared to males and pay less for Personal Auto compares to Males')

gender  policy_type   
F       Corporate Auto    433.738499
        Personal Auto     452.755476
        Special Auto      453.280164
M       Corporate Auto    407.829300
        Personal Auto     457.010178
        Special Auto      429.527942
Name: total_claim_amount, dtype: float64

Females are paying more for corporate auto and special auto policy compared to males and pay less for Personal Auto compares to Males


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [158]:
grouped=insurance_df.groupby(['state','policy'])['customer'].count()
display(grouped[grouped>500])

state       policy     
Arizona     Personal L3     740
California  Personal L2     904
            Personal L3    1249
Oregon      Personal L2     613
            Personal L3    1154
Name: customer, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [159]:
grouped=insurance_df.groupby(['gender','education']).agg({'customer_lifetime_value':['max','min','median']})
display(grouped)
print("The maximum customer lifetime value is lesser among females who have high school or below education compared to females who are educated higher.")
print("The minimum customer lifetime value is lesser among females who have College education compared to females with other education.")

customer_lifetime_value                          
                                                max          min       median
gender education                                                             
F      Bachelor                         73225.95652  1904.000852  5632.613078
       College                          61850.18803  1898.683686  5623.611187
       Doctor                           44856.11397  2395.570000  5332.462694
       High School or Below             55277.44589  2144.921535  6035.086336
       Master                           51016.06704  2417.777032  5729.855012
M      Bachelor                         67907.27050  1898.007675  5548.031892
       College                          61134.68307  1918.119700  6005.847375
       Doctor                           32677.34284  2267.604038  5577.669457
       High School or Below             83325.38119  1940.981221  6286.731006
       Master                           50568.25912  2272.307310  5579.099207

The maximum customer lifetime value is lesser among females who have high school or below education compared to females who are educated higher.
The minimum customer lifetime value is lesser among females who have College education compared to females with other education.


5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [160]:
# Extract month and year from 'effective_to_date' column
insurance_df['effective_to_date'] = pd.to_datetime(insurance_df['effective_to_date'])
insurance_df['month'] = insurance_df['effective_to_date'].dt.month

# Group by state and month, and count the number of policies sold
policies_by_state_and_month = insurance_df.groupby(['state', 'month'])['number_of_policies'].sum().reset_index()

# Pivot the table to arrange months as columns and states as rows
policies_by_state_and_month.pivot_table(index='state', columns='month', values='number_of_policies', fill_value=0)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8752\1788313841.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  insurance_df['effective_to_date'] = pd.to_datetime(insurance_df['effective_to_date'])


month,1,2,3,4,5,6,7,8,9,10,11,12
state,,,,,,,,,,,,
Arizona,2117.0,1768.0,249.0,242.0,213.0,194.0,159.0,207.0,200.0,192.0,160.0,202.0
California,3903.0,3096.0,412.0,282.0,381.0,334.0,390.0,285.0,355.0,377.0,470.0,311.0
Nevada,1043.0,892.0,104.0,75.0,71.0,75.0,96.0,90.0,75.0,110.0,75.0,58.0
Oregon,3202.0,2676.0,277.0,321.0,274.0,276.0,281.0,217.0,299.0,338.0,247.0,210.0
Washington,944.0,777.0,90.0,62.0,100.0,66.0,82.0,67.0,99.0,125.0,79.0,92.0


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [161]:
# Extract month from 'effective_to_date' column
insurance_df['month'] = insurance_df['effective_to_date'].dt.month

# Group by state and month, and sum the number of policies sold
policies_by_state_and_month = insurance_df.groupby(['state', 'month'])['number_of_policies'].sum().reset_index()

# Sort the data by the count of policies sold in descending order
sorted_states = policies_by_state_and_month.groupby('state')['number_of_policies'].sum().sort_values(ascending=False)

# Select the top 3 states with the highest number of policies sold
top_3_states = sorted_states.head(3).index

# Filter the data to include only the top 3 states
top_3_states_data = policies_by_state_and_month[policies_by_state_and_month['state'].isin(top_3_states)]

# Create a new DataFrame that contains the number of policies sold by month for each of the top 3 states
policies_by_month_top_3_states = top_3_states_data.pivot_table(index='state', columns='month', values='number_of_policies', fill_value=0)

print(policies_by_month_top_3_states)

month           1       2      3      4      5      6      7      8      9   \
state                                                                         
Arizona     2117.0  1768.0  249.0  242.0  213.0  194.0  159.0  207.0  200.0   
California  3903.0  3096.0  412.0  282.0  381.0  334.0  390.0  285.0  355.0   
Oregon      3202.0  2676.0  277.0  321.0  274.0  276.0  281.0  217.0  299.0   

month          10     11     12  
state                            
Arizona     192.0  160.0  202.0  
California  377.0  470.0  311.0  
Oregon      338.0  247.0  210.0  
